In [14]:
import os
import pandas as pd
import numpy as np

from implicit.als import AlternatingLeastSquares

## 데이터 준비와 전처리

In [15]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [16]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [17]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [18]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [19]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [20]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [21]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [22]:
# 가장 인기 있는 영화 30개 (인기순)
movie_count = ratings.groupby('movie_id')['user_id'].count()
top_30 = movie_count.sort_values(ascending=False).head(30)
top_30

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [23]:
# top 30 영화 이름으로 변환
movies.set_index('movie_id').loc[top_30.index]

,title,genre
movie_id,,
2858,American Beauty (1999),Comedy|Drama
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
2028,Saving Private Ryan (1998),Action|Drama|War
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
593,"Silence of the Lambs, The (1991)",Drama|Thriller
1198,Raiders of the Lost Ark (1981),Action|Adventure
1270,Back to the Future (1985),Comedy|Sci-Fi


## 선호하는 영화를 5가지 골라서 ratings에 추가

In [24]:
# 좋아하는 영화 설정
my_favorite = [1, 480, 1270, 1580, 2571] #Toy Story (1995), Jurassic Park (1993), Jurassic Park (1993), Men in Black (1997), Matrix, The (1999)

# 'hyunbin'이라는 user_id로 위 영화를 각 5점 평점 준 걸로 가정
my_ratings = pd.DataFrame({'user_id': ['hyunbin']*5, 'movie_id': my_favorite, 'counts':[5]*5})

if not ratings.isin({'user_id':['hyunbin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_ratings)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,hyunbin,1,5,NaN
1,hyunbin,480,5,NaN
2,hyunbin,1270,5,NaN
3,hyunbin,1580,5,NaN
4,hyunbin,2571,5,NaN


In [25]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [26]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['hyunbin'])    # 본인 추가 이전에 6039명이었으니까 6040의 인덱스 6039가 나올 것
print(movie_to_idx[1])

6039
40


In [27]:
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

ratings

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,counts,timestamp
0,0,0,5,978300760.0
1,0,1,3,978302109.0
2,0,2,3,978301968.0
3,0,3,4,978300275.0
4,0,4,5,978824291.0
...,...,...,...,...
0,6039,40,5,NaN
1,6039,107,5,NaN
2,6039,22,5,NaN
3,6039,175,5,NaN


## CSR matrix 만들기

In [28]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## Alternating Least Squares 모델 구성 및 훈련

In [29]:
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [30]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [31]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [32]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 훈련된 모델이 예측한 본인의 Toy Story에 대한 선호도 파악

In [33]:
hyunbin, toy_story = user_to_idx['hyunbin'], movie_to_idx[1]
hyunbin_vector, toy_story_vector = als_model.user_factors[hyunbin], als_model.item_factors[toy_story]

In [34]:
hyunbin_vector

array([ 3.0526713e-01,  2.6737204e-01,  2.3844313e-02, -3.0975351e-01,
       -5.4538268e-01, -3.6703387e-01,  1.7576689e-01,  6.2348717e-01,
        1.6261101e-01,  6.1628121e-01, -3.1892407e-01, -8.9119035e-01,
       -1.5169543e+00,  4.7579864e-01,  1.7684609e-01, -4.2423299e-01,
        9.3956918e-02, -6.0671252e-01, -2.0008138e-01, -3.3988118e-01,
        1.1839864e+00,  4.1581395e-01,  6.4998412e-01, -6.9521815e-02,
       -1.7615809e-01, -7.6197231e-01, -5.5511147e-02, -1.0064313e+00,
       -3.0087781e-01,  4.3479145e-01,  7.2771049e-01,  9.3599099e-01,
        1.3794887e+00,  1.2603992e-01, -9.7961947e-02, -3.8346095e-04,
       -6.9673067e-01,  1.7222157e-02, -2.7121367e-02, -4.5155975e-01,
       -5.2908003e-01, -2.7592751e-01, -8.2448363e-01,  1.0148895e+00,
        1.6352259e-01, -8.4904835e-02, -5.4236609e-01,  1.6118172e-01,
       -1.9803463e-01, -1.8728784e-01,  1.5045312e-01, -5.6843674e-01,
       -1.1371298e+00,  1.8429066e-01,  1.1205829e+00,  2.1021569e-01,
      

In [35]:
toy_story_vector

array([-2.05576140e-02, -1.43294325e-02,  1.60410572e-02, -3.23378411e-03,
        1.34483995e-02, -8.22228007e-03,  2.35349163e-02,  3.54105718e-02,
        6.44504093e-03,  2.72280835e-02, -4.84372629e-03, -1.01495171e-02,
       -3.02909613e-02,  2.41133254e-02,  1.85390543e-02,  6.50967378e-03,
        2.97425743e-02,  1.86117738e-02,  1.89317185e-02, -3.79939675e-02,
        1.64117441e-02,  2.26284154e-02,  4.24644537e-03, -2.22780239e-02,
       -1.95378959e-02, -2.07938496e-02, -4.93112858e-03, -1.54466205e-03,
       -7.73848360e-03,  3.11337486e-02,  1.48021812e-02,  1.02535486e-02,
        2.57983338e-02,  6.14551982e-06, -2.85718404e-02, -2.89980005e-02,
        3.60662676e-03, -3.36072152e-03,  1.10290144e-02,  3.27389315e-03,
        1.12881605e-02, -1.25727430e-02, -1.05529197e-03,  2.20148228e-02,
        9.87285934e-03,  1.18541364e-02, -1.28400596e-02,  1.43886413e-02,
        1.44833524e-03, -1.56511497e-02,  3.41982371e-03, -1.08458241e-02,
       -2.18228623e-02, -

In [36]:
np.dot(hyunbin_vector, toy_story_vector)

0.47527733

### factors를 증가시켜 모델을 다시 훈련

In [38]:
# factors 늘려서 다시 훈련
als_model = AlternatingLeastSquares(factors=250, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [39]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [40]:
hyunbin, toy_story = user_to_idx['hyunbin'], movie_to_idx[1]
hyunbin_vector, toy_story_vector = als_model.user_factors[hyunbin], als_model.item_factors[toy_story]

In [41]:
np.dot(hyunbin_vector, toy_story_vector)

0.74208283

## 좋아하는 영화와 비슷한 영화 추천받기

In [42]:
movie_to_title = movies.set_index('movie_id')['title'].to_dict()
movie_to_title

{1: 'Toy Story (1995)',
 2: 'Jumanji (1995)',
 3: 'Grumpier Old Men (1995)',
 4: 'Waiting to Exhale (1995)',
 5: 'Father of the Bride Part II (1995)',
 6: 'Heat (1995)',
 7: 'Sabrina (1995)',
 8: 'Tom and Huck (1995)',
 9: 'Sudden Death (1995)',
 10: 'GoldenEye (1995)',
 11: 'American President, The (1995)',
 12: 'Dracula: Dead and Loving It (1995)',
 13: 'Balto (1995)',
 14: 'Nixon (1995)',
 15: 'Cutthroat Island (1995)',
 16: 'Casino (1995)',
 17: 'Sense and Sensibility (1995)',
 18: 'Four Rooms (1995)',
 19: 'Ace Ventura: When Nature Calls (1995)',
 20: 'Money Train (1995)',
 21: 'Get Shorty (1995)',
 22: 'Copycat (1995)',
 23: 'Assassins (1995)',
 24: 'Powder (1995)',
 25: 'Leaving Las Vegas (1995)',
 26: 'Othello (1995)',
 27: 'Now and Then (1995)',
 28: 'Persuasion (1995)',
 29: 'City of Lost Children, The (1995)',
 30: 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 31: 'Dangerous Minds (1995)',
 32: 'Twelve Monkeys (1995)',
 33: 'Wings of Courage (1995)',
 34: 'Babe (1

In [43]:
favorite_movie = 480  # Jurassic Park
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(107, 1.0),
 (175, 0.50090283),
 (82, 0.36762768),
 (92, 0.3617919),
 (3151, 0.3306642),
 (2748, 0.3282018),
 (3137, 0.3222412),
 (3482, 0.31793702),
 (2630, 0.31772903),
 (2736, 0.316678),
 (2994, 0.31565234),
 (3025, 0.3155635),
 (3289, 0.31485674),
 (3265, 0.31475118),
 (1343, 0.3145648)]

In [44]:
#movie_to_idx 를 뒤집어, index로부터 movie_id를 얻는 dict를 생성 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[movie_to_title[idx_to_movie[i[0]]] for i in similar_movie]

['Jurassic Park (1993)',
 'Men in Black (1997)',
 'Lost World: Jurassic Park, The (1997)',
 'Terminator 2: Judgment Day (1991)',
 'And God Created Woman (Et Dieu&#8230;Créa la Femme) (1956)',
 'Bewegte Mann, Der (1994)',
 "Greaser's Palace (1972)",
 "Pot O' Gold (1941)",
 'Midaq Alley (Callejón de los milagros, El) (1995)',
 'Nénette et Boni (1996)',
 'Big Country, The (1958)',
 'Abominable Snowman, The (1957)',
 'Nekromantik (1987)',
 'Quiet Room, The (1996)',
 'Little Lord Fauntleroy (1936)']

In [45]:
user = user_to_idx['hyunbin']
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(92, 0.4440867),
 (50, 0.24486664),
 (110, 0.24066922),
 (38, 0.225351),
 (150, 0.22391786),
 (479, 0.21885015),
 (141, 0.21620508),
 (62, 0.2068093),
 (44, 0.19765028),
 (375, 0.17924072),
 (145, 0.1750086),
 (82, 0.17029728),
 (87, 0.15372042),
 (13, 0.14496846),
 (356, 0.14211138),
 (768, 0.13861844),
 (674, 0.13612854),
 (670, 0.13056113),
 (160, 0.12996054),
 (200, 0.12992871)]

In [46]:
[movie_to_title[idx_to_movie[i[0]]] for i in movie_recommended]

['Terminator 2: Judgment Day (1991)',
 'Toy Story 2 (1999)',
 'Groundhog Day (1993)',
 'Sixth Sense, The (1999)',
 'Independence Day (ID4) (1996)',
 'Contact (1997)',
 'Fugitive, The (1993)',
 'Total Recall (1990)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Face/Off (1997)',
 'Fifth Element, The (1997)',
 'Lost World: Jurassic Park, The (1997)',
 'Braveheart (1995)',
 "Ferris Bueller's Day Off (1986)",
 'Legends of the Fall (1994)',
 'Get Shorty (1995)',
 'Back to the Future Part II (1989)',
 'Galaxy Quest (1999)',
 'Forrest Gump (1994)',
 'Terminator, The (1984)']

In [50]:
# X-Men 추천에 기영한 영화의 기여 정도 확인
[movie_to_title[idx_to_movie[i[0]]] for i in movie_recommended].index('Toy Story 2 (1999)')

1

In [51]:
movie_recommended[11]

(82, 0.17029728)

In [52]:
xmen = 372
explain = als_model.explain(user, csr_data, itemid=xmen)

In [53]:
[(movie_to_title[idx_to_movie[i[0]]], i[1]) for i in explain[1]]

[('Matrix, The (1999)', 0.09114289036966247),
 ('Men in Black (1997)', 0.046348033090963726),
 ('Back to the Future (1985)', 0.006581311378708153),
 ('Jurassic Park (1993)', -0.005644099009596483),
 ('Toy Story (1995)', -0.04320590729257605)]